<a href="https://colab.research.google.com/github/asleybach/googlecolab-notebooks/blob/main/client_Oakdb_check_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script to connect to the consumption DB

## 1 Read metadata


In [ ]:
pip install psycopg2-binary

In [ ]:
import psycopg2
import pandas as pd
from getpass import getpass

insert username and corresponding password

In [ ]:
_username = getpass('Enter the username value: ')
_password = getpass('Enter password:')

Enter the username value: ··········
Enter password:··········


In [ ]:
# Database class for management and connection with the database

class Database:
    def __init__(self, db, username, password, port, host):
        self.db = db
        self.username = username
        self.password = password
        self.port = port
        self.host = host
        self.cur = None
        self.conn = None

    def connect(self):
        self.conn = psycopg2.connect(
          database=self.db,
          user=self.username,
          password=self.password,
          host=self.host,
          port=self.port
          )
        self.cur = self.conn.cursor()

    def execute_query(self, query):
        self.cur.execute(query)
        self.conn.commit()

    def get_data(self):
        self.cur.fetchall()

    def close(self):
        self.cur.close()
        self.conn.close()

MyDb = Database(
    db='consumption',
    username=_username,
    password=_password,
    host='oak-consumption.ceedji5fvimj.eu-west-2.rds.amazonaws.com',
    port='5432'
)

In [ ]:
# function to get the resources metadata

def get_metadata():
    MyDb.connect()
    print('** connected to the db consumption_resourceconsumption')
    postgreSQL_select_Query = "SELECT * FROM consumption_metadata"
    MyDb.execute_query(postgreSQL_select_Query)
    df = pd.DataFrame(
        MyDb.cur,
        columns = [
            'id',
            'veId',
            've_name',
            've_active',
            've_city',
            've_contry',
            'postalCode',
            'resourceId',
            'resource_name',
            'resource_description',
            'resource_active',
            'resource_classifier',
            'baseUnit'
        ]
    )
    return df

In [ ]:
# Get metadata

data = get_metadata()

print(data)
#print(data.iloc[0])
#print(data['resource_active'])



** connected to the db consumption_resourceconsumption
     id                                  veId  ... resource_classifier  baseUnit
0    40  0577bf7d-1ce9-4ddc-b783-18abca7b1fd7  ...                None      None
1    41  0577bf7d-1ce9-4ddc-b783-18abca7b1fd7  ...                None      None
2    42  0577bf7d-1ce9-4ddc-b783-18abca7b1fd7  ...                None      None
3    17  94c6c61f-8113-4807-8163-958b832ad181  ...          MONITORING       Kwh
4    18  94c6c61f-8113-4807-8163-958b832ad181  ...          MONITORING       Kwh
..  ...                                   ...  ...                 ...       ...
84  102  c07c0e42-9712-4b49-a905-5873e43cac8a  ...          MONITORING       Kwh
85  103  c07c0e42-9712-4b49-a905-5873e43cac8a  ...          ALWAYS_OFF       Kwh
86  104  906949cc-2d9c-4530-adef-eb09c73dad55  ...          ALWAYS_OFF       Kwh
87  105  15f84176-616c-43e4-9f42-ae800777c30d  ...                None      None
88   90  15f84176-616c-43e4-9f42-ae800777c30d  ...    

In [ ]:
df = pd.DataFrame(data, columns = ['resource_name','resource_active'])
df

,resource_name,resource_active
0,None,None
1,None,None
2,None,None
3,Railway Barn,True
4,Railway Barn,True
...,...,...
84,Meter,True
85,Signal bridge,True
86,The Studio fridge LCC Ox tall boy,True
87,None,None


In [ ]:
def read_metadata_by_resource(resource_identifier):
    return (data[(data['resourceId']==resource_identifier)])


Test, we use the resource_identifier 51ce905c-fa11-47c0-81c2-7dd519331bd8


In [ ]:
show = read_metadata_by_resource('a099100a-9da4-49d4-b2f3-e1ee7dcc986d')
show

,id,veId,ve_name,ve_active,ve_city,ve_contry,postalCode,resourceId,resource_name,resource_description,resource_active,resource_classifier,baseUnit
88,90,15f84176-616c-43e4-9f42-ae800777c30d,Mr. Fogg's House of Botanicals,True,"Fitzrovia, London",None,E2 68CB,a099100a-9da4-49d4-b2f3-e1ee7dcc986d,None,None,None,None,None


## 2 Returns consumption data for a specific resource

insert the identifier of the resource to see its consumption data,
change to check another resource

In [ ]:
_resource = 'a099100a-9da4-49d4-b2f3-e1ee7dcc986d'

In [ ]:
def data_by_resource(resource):
  MyDb.connect()
  postgreSQL_select_Query = f"SELECT * FROM consumption_resourceconsumption WHERE reference_resource LIKE '{_resource}'"
  MyDb.execute_query(postgreSQL_select_Query)
  dataf = pd.DataFrame(
          MyDb.cur,
          columns = [
              'id',
              'timestamp',
              'consumption',
              'reference_resource',
          ]
      )

  if dataf.empty:
    print('data not found...')
  return dataf

dataf = data_by_resource(_resource)
dataf

,id,timestamp,consumption,reference_resource
0,410996,2020-11-05 12:56:00,0.0205,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
1,410998,2020-11-05 12:57:00,0.0205,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
2,411000,2020-11-05 12:58:00,0.0206,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
3,411002,2020-11-05 12:59:00,0.0206,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
4,411004,2020-11-05 13:00:00,0.0207,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
...,...,...,...,...
31917,552751,2020-12-24 12:19:00,0.011,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
31918,552752,2020-12-24 12:20:00,0.0109,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
31919,552753,2020-12-24 12:21:00,0.011,a099100a-9da4-49d4-b2f3-e1ee7dcc986d
31920,552754,2020-12-24 12:22:00,0.0109,a099100a-9da4-49d4-b2f3-e1ee7dcc986d


## 3 Returns resources for client

In [ ]:
client_df = pd.DataFrame(data, columns = ['ve_name','resource_name','resource_active'])
client_df

,ve_name,resource_name,resource_active
0,Sager & Wilde - Paradise Row,None,None
1,Sager & Wilde - Paradise Row,None,None
2,Sager & Wilde - Paradise Row,None,None
3,New Vineyard,Railway Barn,True
4,New Vineyard,Railway Barn,True
...,...,...,...
84,LCC Shaftesbury,Meter,True
85,LCC Shaftesbury,Signal bridge,True
86,LCC Oxford Street,The Studio fridge LCC Ox tall boy,True
87,Mr. Fogg's House of Botanicals,None,None


**Return all resources by a client**

To search for resources by user registered on the platform, the user name must be assigned, are examples of user names


*   Sager and Wilde Paradise Row
*   LCC Oxford Circus
*   Queen Park Cafe
*   The Elgin
*   Coin Laudry
*   Heads & Tails
*   Victory Mansion
*   LCC Shaftesbury







In [ ]:
_member_name = 'Sager & Wilde - Paradise Row'

In [ ]:
resources_client_df = client_df.loc[client_df.ve_name == _member_name]
if resources_client_df.empty:
  print('not found')
resources_client_df

,ve_name,resource_name,resource_active
0,Sager & Wilde - Paradise Row,None,None
1,Sager & Wilde - Paradise Row,None,None
2,Sager & Wilde - Paradise Row,None,None
30,Sager & Wilde - Paradise Row,None,None
